In [5]:
source('~/canonades/RcisTarget/src/RCTpipe.r', echo = T, verbose = T)

'envir' chosen:<environment: R_GlobalEnv>
encoding = "native.enc" chosen
 --> from_file='TRUE'
 lines:function (x, ...)  
--> parsed 32 expressions; now eval(.)ing them:

>>>> eval(expression_nr. 1 )

> Sys.setenv(LOG_LEVEL = "TRACE")
curr.fun: symbol Sys.setenv
 .. after ‘expression(Sys.setenv(LOG_LEVEL = "TRACE"))’

>>>> eval(expression_nr. 2 )

> Sys.getenv("R_HOME")
curr.fun: symbol Sys.getenv
[1] "/users/genomics/xoel/canonades/RcisTarget/conda_env/lib/R"
 .. after ‘expression(Sys.getenv("R_HOME"))’

>>>> eval(expression_nr. 3 )

> PAR_METHOD <- "iCisTarget"
curr.fun: symbol <-
 .. after ‘expression(PAR_METHOD <- "iCisTarget")’

>>>> eval(expression_nr. 4 )

> PAR_SEARCH_RANGE <- "long"
curr.fun: symbol <-
 .. after ‘expression(PAR_SEARCH_RANGE <- "long")’

>>>> eval(expression_nr. 5 )

> PAR_MOTIF_NES <- 3
curr.fun: symbol <-
 .. after ‘expression(PAR_MOTIF_NES <- 3)’

>>>> eval(expression_nr. 6 )

> PAR_MOTIF_DB_V <- "V10"
curr.fun: symbol <-
 .. after ‘expression(PAR_MOTIF_DB_V

### Parameters, files and dirs

In [6]:
# Input file
input.fname <- 'data/nico_peak_expression.csv'

# Results directory
results.dir <- './results/RCT_peaks/'

In [7]:
# Number of cores
n.cores <- parallel::detectCores()
# Number of permutations
n.perms <- 1000

In [8]:
# Expression data for bicorrelations
exp.fname <- 'data/nico_expression_log2.onlyExpressed.csv'
# exp.fname <- 'data/nico_expression_log2.original_genes.csv'
# Disease data for bicorrelations
dis.fname <- 'data/gene_disorder_associations.intersectionExpData.csv'


# Data

In [9]:
peaks.df <- read.csv(input.fname, row.names=1)
dim(peaks.df)
head(peaks.df)

[1] 19735    20

,P2.FGF20,P2.FGF10,P2.FGF1,P2.FGF0.1,P3.FGF20,P3.FGF10,P3.FGF1,P3.FGF0.1,P4.FGF20,P4.FGF10,P4.FGF1,P4.FGF0.1,P6.FGF20,P6.FGF10,P6.FGF1,P6.FGF0.1,P8.FGF20,P8.FGF10,P8.FGF1,P8.FGF0.1
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
RPPH1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
RMRP,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
EEF1A1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
RPS27,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
HIST1H4E,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
RPL23,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [10]:
exp.data <- read.csv(exp.fname, row.names=1)
dim(exp.data)
head(exp.data)

[1] 19735    20

,P2.FGF20,P2.FGF10,P2.FGF1,P2.FGF0.1,P3.FGF20,P3.FGF10,P3.FGF1,P3.FGF0.1,P4.FGF20,P4.FGF10,P4.FGF1,P4.FGF0.1,P6.FGF20,P6.FGF10,P6.FGF1,P6.FGF0.1,P8.FGF20,P8.FGF10,P8.FGF1,P8.FGF0.1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RPPH1,13.04623,12.28241,11.24364,12.27173,12.12812,12.03452,13.00799,12.17872,12.05932,11.86173,11.98733,11.62126,12.00351,12.63748,11.93306,12.33148,12.23537,12.43607,12.59975,12.52714
RMRP,12.48491,12.11243,11.38070,12.09172,12.34055,12.44420,13.10532,12.28078,12.08826,11.46426,11.66939,11.90056,11.91875,11.97331,11.51441,11.98859,11.76499,12.19002,12.34510,12.08683
EEF1A1,11.62933,11.59329,11.66738,11.63242,11.68348,11.69902,11.56305,11.34799,11.76610,11.79932,11.75743,11.05350,11.49185,11.69776,11.58537,11.67830,11.39488,11.75448,11.73382,11.78609
RPS27,10.51746,11.34083,11.32416,11.54350,11.45687,11.54094,11.09534,11.39159,11.34469,11.30040,11.33700,10.87653,11.00994,10.38989,11.26149,11.32254,10.69421,11.16709,11.29963,11.25599
HIST1H4E,11.86527,10.61179,10.55470,10.90878,11.08158,10.89391,11.72504,10.57039,11.11413,10.92014,11.03981,10.98338,11.24872,12.17598,10.92614,10.78664,10.62988,10.98815,10.69400,10.27310
RPL23,11.09727,11.02227,11.07019,11.00418,10.98213,10.96352,10.80222,10.86794,10.96422,11.07841,11.12587,10.65969,10.70524,11.06947,11.07815,10.82535,10.72543,10.96961,10.94586,10.90868


In [11]:
diseases.df <- read.csv(dis.fname, row.names=1)[rownames(peaks.df),]
dim(diseases.df)
head(diseases.df)

[1] 19735    19

,Microcephaly,Hydrocephaly,RareMCD,MDD_2018,Polymicrogyria,SCZ_2020,AN_2019,NEUROT_2018,PD_2014,AD_2019,Lissencephaly,BD_2019,DevDyslexia,Heterotopia,ADHD_2019,DD,FCDandmTOR,Cobblestone,ASD
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
RPPH1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
RMRP,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
EEF1A1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
RPS27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
HIST1H4E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
RPL23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Run

### Regulons

In [ ]:
RCT <- run_pipe(
    genes.df = peaks.df,
    ncores = n.cores
)

In [9]:
split(RCT$summaryTbl$TF_highConf, RCT$summaryTbl$geneSet)

$P2.FGF1
[1] "OVOL1"

$P2.FGF10
[1] "ZNF667"

$P2.FGF20
[1] "THAP11"

$P3.FGF0.1
[1] "ATF4"  "ZBTB6"

$P3.FGF10
[1] "DMRT1"

$P4.FGF0.1
 [1] "BCL6B"   "DIDO1"   "E2F3"    "GLIS2"   "ILF3"    "KLF7"    "KLF8"   
 [8] "MECP2"   "MTF1"    "PLAGL1"  "RBAK"    "SIN3A"   "SMAD9"   "SP1"    
[15] "SP2"     "SREBF2"  "TAF1"    "TFDP1"   "ZBTB10"  "ZBTB5"   "ZBTB8A" 
[22] "ZFHX2"   "ZNF142"  "ZNF37A"  "ZNF383"  "ZNF398"  "ZNF460"  "ZNF519" 
[29] "ZNF704"  "ZNF846"  "ZSCAN22" "ZXDB"    "ZXDC"   

$P4.FGF10
[1] "PLAG1"

$P6.FGF0.1
[1] "MEIS1" "SOX4" 

$P6.FGF10
[1] "GMEB1"

$P6.FGF20
[1] "NR1H4"

$P8.FGF0.1
[1] "PURA"

$P8.FGF1
[1] "CEBPD" "PITX3" "VSX2" 

$P8.FGF10
[1] "PRDM9"

$P8.FGF20
 [1] "EGR1"   "EGR2"   "ETV5"   "KLF10"  "KLF17"  "KLF3"   "NR3C1"  "SP6"   
 [9] "YY2"    "ZBTB37" "ZNF148"

## Odds-ratios

In [10]:
RCT[['odds.ratio.diseases']] <- odds.ratio.RCT(RCT, df_to_test = diseases.df, haldane_correction = T)

2023-01-26 18:00:14 [INFO] Number of genes not tested in RcisTarget: 1840
2023-01-26 18:00:14 [INFO] P2.FGF1: 1113
2023-01-26 18:00:14 [INFO] Regulon core OVOL1: 199
2023-01-26 18:00:14 [INFO] 	Testing Microcephaly: 64
2023-01-26 18:00:14 [INFO] 	Testing Hydrocephaly: 105
2023-01-26 18:00:14 [INFO] 	Testing RareMCD: 23


Using Haldane correction



2023-01-26 18:00:14 [INFO] 	Testing MDD_2018: 51
2023-01-26 18:00:14 [INFO] 	Testing Polymicrogyria: 28


Using Haldane correction



2023-01-26 18:00:14 [INFO] 	Testing SCZ_2020: 108
2023-01-26 18:00:14 [INFO] 	Testing AN_2019: 99
2023-01-26 18:00:14 [INFO] 	Testing NEUROT_2018: 268
2023-01-26 18:00:14 [INFO] 	Testing PD_2014: 29
2023-01-26 18:00:14 [INFO] 	Testing AD_2019: 175
2023-01-26 18:00:14 [INFO] 	Testing Lissencephaly: 40


...

2023-01-26 18:00:21 [INFO] 	Testing AD_2019: 175
2023-01-26 18:00:21 [INFO] 	Testing Lissencephaly: 40
2023-01-26 18:00:21 [INFO] 	Testing BD_2019: 194
2023-01-26 18:00:21 [INFO] 	Testing DevDyslexia: 24
2023-01-26 18:00:21 [INFO] 	Testing Heterotopia: 13


Using Haldane correction



2023-01-26 18:00:21 [INFO] 	Testing ADHD_2019: 26
2023-01-26 18:00:21 [INFO] 	Testing DD: 90
2023-01-26 18:00:21 [INFO] 	Testing FCDandmTOR: 64
2023-01-26 18:00:21 [INFO] 	Testing Cobblestone: 14


Using Haldane correction



2023-01-26 18:00:21 [INFO] 	Testing ASD: 965


In [11]:
save_RCT(RCT, save_dir = results.dir)

2023-01-26 18:00:21 [INFO] Saving RCT to ./results/RCT_peaks/
2023-01-26 18:00:21 [INFO] Saving results without permutations


# Supplementary table 5

In [1]:
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
RCT <- readRDS('results/RCT_peaks/cisTarget_results.rds')

In [19]:
dfs <- list(
    'Temporal regulons' = RCT$regulonTbl,
    'Disease enrichment' = RCT$odds.ratio.diseases %>% select(
        geneSet, Core, testSet, 
        `estimate.odds ratio`, OR.log2, p.value, conf.int.min, conf.int.max,
        `REG TEST`, `REG NOT TEST`, `NOT REG TEST`, `NOT REG NOT TEST`) %>% arrange(p.value)
)

In [20]:
merge(dfs[[1]], diseases.df, by.x='Target', by.y='row.names', all.x=T, all.y=F) %>%
    select_at(c(colnames(dfs[[1]]), colnames(diseases.df))) %>%
    mutate_at(.vars = colnames(diseases.df), function(X){ifelse(is.na(X), 0, X)}) -> dfs[[1]]

In [21]:
split(dfs[[1]], dfs[[1]]$Core) %>% lapply(function(df){
    sapply(colnames(diseases.df), function(dis){
        paste0(c(df[as.logical(df[,dis]),'Target'], recursive=T), collapse=',')
    }) ->  enriched.targets
    data.frame(Risk_Targets=enriched.targets, testSet=colnames(diseases.df))
}) -> all.enriched.targets

In [22]:
for (core in names(all.enriched.targets)){
    all.enriched.targets[[core]]$Core <- core
}
all.enriched.targets %>% do.call(what='rbind') -> all.enriched.targets

In [23]:
merge(dfs[[2]], all.enriched.targets, by=c('Core', 'testSet'), all=T) -> dfs[[2]]

In [24]:
dfs[[1]] %>% mutate(geneSet=factor(geneSet, levels=colnames(peaks.df))) %>% arrange(geneSet, Core) -> dfs[[1]]
dfs[[2]] %>% mutate(conf.int.min=NULL,
                    conf.int.max=NULL, 
                    geneSet=factor(geneSet, levels=colnames(peaks.df)),
                    testSet=factor(testSet, levels=colnames(diseases.df))) %>% select(geneSet, everything()) %>% arrange(geneSet, Core, testSet) -> dfs[[2]]

In [25]:
# Add column of enriched targets

In [26]:
dfs[[2]]

geneSet,Core,testSet,estimate.odds ratio,OR.log2,p.value,REG TEST,REG NOT TEST,NOT REG TEST,NOT REG NOT TEST,Risk_Targets
<fct>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
P2.FGF20,THAP11,Microcephaly,0.4624706,-1.11256641,1.00000000,0,145,9,1278,
P2.FGF20,THAP11,Hydrocephaly,1.1101331,0.15073261,0.61850881,1,144,8,1279,RNASEH2A
P2.FGF20,THAP11,RareMCD,2.9473081,1.55939790,1.00000000,0,145,1,1286,
P2.FGF20,THAP11,MDD_2018,2.9473081,1.55939790,1.00000000,0,145,1,1286,
P2.FGF20,THAP11,Polymicrogyria,1.7670103,0.82131046,1.00000000,0,145,2,1285,
P2.FGF20,THAP11,SCZ_2020,5.4054199,2.43440668,0.03889343,3,142,5,1282,"BCL2L12,ELAC2,PAK6"
P2.FGF20,THAP11,AN_2019,2.6951806,1.43038196,0.13682837,3,142,10,1277,"CDC25A,CYB561D2,RBM5"
P2.FGF20,THAP11,NEUROT_2018,0.7361566,-0.44191547,0.75792001,2,143,24,1263,"CLP1,GPANK1"
P2.FGF20,THAP11,PD_2014,8.8487973,3.14548137,1.00000000,0,145,0,1287,


In [27]:
openxlsx::write.xlsx(x = dfs, file = 'results/supp_table.5.Temporal_regulons_with_disease_enrichment.xlsx')

### Core-Target bicor

In [13]:
RCT[['bicorRes']] <- compute_bicors(
    RCT_result=RCT,
    exp.data=exp.data,
    n.permutations = n.perms,
    ncores=n.cores)

2023-01-14 01:50:18 [DEBUG] [BICOR] geneSet P2.FGF1
2023-01-14 01:50:18 [DEBUG] [BICOR]    Core OVOL1
2023-01-14 01:50:18 [TRACE] Core in expression data: TRUE
2023-01-14 01:50:18 [TRACE] Targets not in expression data: 0




Warning message in bicor(x, y, use = use, ...):
“bicor: zero MAD in variable 'x'. Pearson correlation was used for individual columns with zero (or missing) MAD.”
Warning message in bicor(x, y, use = use, ...):
“bicor: zero MAD in variable 'y'. Pearson correlation was used for individual columns with zero (or missing) MAD.”
Loading required package: doParallel

Loading required package: foreach

Loading required package: iterators

Loading required package: parallel



2023-01-14 01:51:21 [DEBUG] [BICOR] geneSet P2.FGF10
2023-01-14 01:51:21 [DEBUG] [BICOR]    Core ZNF667
2023-01-14 01:51:21 [TRACE] Core in expression data: TRUE
2023-01-14 01:51:21 [TRACE] Targets not in expression data: 0


Warning message in bicor(x, y, use = use, ...):
“bicor: zero MAD in variable 'y'. Pearson correlation was used for individual columns with zero (or missing) MAD.”


2023-01-14 01:52:22 [DEBUG] [BICOR] geneSet P2.FGF20
2023-01-14 01:52:22 [DEBUG] [BICOR]    Core THAP11
2023-01-14 01:52:22 [TRACE] Core in expression data: TRUE
2023-01-14 01:52:22 [TRACE] Targets not in expression data: 0


...

Warning message in bicor(x, y, use = use, ...):
“bicor: zero MAD in variable 'y'. Pearson correlation was used for individual columns with zero (or missing) MAD.”


2023-01-14 02:50:30 [DEBUG] [BICOR]    Core ZBTB37
2023-01-14 02:50:30 [TRACE] Core in expression data: TRUE
2023-01-14 02:50:30 [TRACE] Targets not in expression data: 0


Warning message in bicor(x, y, use = use, ...):
“bicor: zero MAD in variable 'y'. Pearson correlation was used for individual columns with zero (or missing) MAD.”


2023-01-14 02:51:31 [DEBUG] [BICOR]    Core ZNF148
2023-01-14 02:51:31 [TRACE] Core in expression data: TRUE
2023-01-14 02:51:31 [TRACE] Targets not in expression data: 0


Warning message in bicor(x, y, use = use, ...):
“bicor: zero MAD in variable 'y'. Pearson correlation was used for individual columns with zero (or missing) MAD.”


In [14]:
save_RCT(RCT, save_dir = results.dir)

2023-01-14 02:53:37 [INFO] Saving RCT to ./results/RCT_peaks/
2023-01-14 02:53:37 [INFO] Saving results without permutations
2023-01-14 02:57:03 [INFO] Saving results of bicorrelations


## Permutations of cores

In [ ]:
RCT <- readRDS('results/RCT_peaks/cisTarget_results.rds')

In [ ]:
n.perms

In [ ]:
RCT.perm <- permute_RCT(
    RCT,
    n.permutations = n.perms
    # n.permutations = 10
)

In [ ]:
save_RCT(RCT.perm, save_dir = results.dir)

In [20]:

saved <- readRDS('results/RCT_peaks/cisTarget_results.permuted.rds')

In [21]:
saved$permutations$perm.cores

geneSet,TF_highConf,TFperm_counts,TFperm_pval.nom,TFperm_pval.adj,TFperm_pval.adj.geneSet
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>
P2.FGF1,ACAA1,7,0.007,0.01873537,0.01739212
P2.FGF1,ADNP,2,0.002,0.01653948,0.01325114
P2.FGF1,ADNP2,7,0.007,0.01873537,0.01739212
P2.FGF1,AEBP2,7,0.007,0.01873537,0.01739212
P2.FGF1,AFF4,1,0.001,0.01653948,0.01250862
P2.FGF1,AHCTF1,1,0.001,0.01653948,0.01250862
P2.FGF1,AHDC1,2,0.002,0.01653948,0.01325114
P2.FGF1,AHR,17,0.017,0.02468931,0.02324882
P2.FGF1,ALX1,7,0.007,0.01873537,0.01739212


In [22]:
RCT.perm$permutations$perm.cores

geneSet,TF_highConf,TFperm_counts,TFperm_pval.nom,TFperm_pval.adj,TFperm_pval.adj.geneSet
<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>
P2.FGF1,ACAA1,7,0.007,0.01873537,0.01739212
P2.FGF1,ADNP,2,0.002,0.01653948,0.01325114
P2.FGF1,ADNP2,7,0.007,0.01873537,0.01739212
P2.FGF1,AEBP2,7,0.007,0.01873537,0.01739212
P2.FGF1,AFF4,1,0.001,0.01653948,0.01250862
P2.FGF1,AHCTF1,1,0.001,0.01653948,0.01250862
P2.FGF1,AHDC1,2,0.002,0.01653948,0.01325114
P2.FGF1,AHR,17,0.017,0.02468931,0.02324882
P2.FGF1,ALX1,7,0.007,0.01873537,0.01739212


In [ ]:
RCT$